## 데이터 구조

```text
dataSet
├── info                        ← 면접 메타데이터
│   ├── date                    : 날짜 (YYYYMMDD)
│   ├── occupation              : 직무 코드 (예: BM, SM 등)
│   ├── channel                 : 인터뷰 방식 (MOCK, REAL 등)
│   ├── place                   : 장소 (ONLINE, OFFLINE 등)
│   ├── gender                  : 성별 (MALE / FEMALE)
│   ├── ageRange                : 연령대 (예: "45-54")
│   └── experience              : 경력 구분 (EXPERIENCED / INEXPERIENCED)
│
├── question                    ← 질문 파트
│   ├── raw                     ← 질문 원문
│   │   ├── text                : 질문 내용 (문자열)
│   │   └── wordCount           : 단어 수 (정수)
│   ├── emotion                 : 질문자의 감정 태그 목록 (리스트)
│   │   └── [0] object
│   │        ├── text           : 감정이 나타난 문장
│   │        ├── expression     : 감정 코드 (예: p-happy, n-nervous)
│   │        └── category       : 감정 범주 (positive / negative / neutral)
│   └── intent                  : 질문 의도 태그 목록 (리스트)
│        └── [0] object
│             ├── text          : 의도 추출 대상 문장
│             ├── expression    : 의도 라벨 (예: c_person)
│             └── category      : 의도 분류 카테고리 (예: background)
│
└── answer                      ← 답변 파트
    ├── raw                     ← 답변 원문
    │   ├── text                : 지원자의 답변 내용 (문자열)
    │   └── wordCount           : 단어 수 (정수)
    ├── emotion                 : 답변 감정 태그 목록 (리스트)
    │   └── [0] object
    │        ├── text           : 감정이 나타난 문장
    │        ├── expression     : 감정 코드 (예: p-happy, n-sad)
    │        └── category       : 감정 범주 (positive / negative / neutral)
    ├── intent                  : 답변 의도 태그 목록 (리스트)
    │   └── [0] object
    │        ├── text           : 의도 추출 대상 문장
    │        ├── expression     : 의도 라벨 (예: c_person)
    │        └── category       : 의도 분류 카테고리 (예: background)
    └── summary                 ← 답변 요약
        ├── text                : 요약 문장
        └── wordCount           : 요약 단어 수


```

## 결측치 찾기 
    - 훈련시킬 수 있는 데이터셋 확인 : 13000개 가량의 emotion데이터가 채워져있음

In [ ]:
import os, json
from collections import defaultdict

training_path = "G:/내 드라이브/면접VR AI제작/extracted_TrainingData"
valid_path = "G:/내 드라이브/면접VR AI제작/extracted_Valid"

# question의 emotion이 비어있거나
# answer의 emotion이 비어있거나
def emotion_empty(base_path) : 
    emotion_count = 0
    emotion_empty_count = 0
    # trainingData의 폴더들을 읽는다.
    for folder_name in os.listdir(base_path) : 
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.isdir(folder_path) :
            continue
        print(f"{folder_name}을 시작합니다.")
        # folder내 .json파일들을 읽는다.
        try : 
            for file_name in os.listdir(folder_path) : 
                if not file_name.endswith(".json") : 
                    continue
                file_path = os.path.join(folder_path, file_name)
                
                with open(file_path, 'r', encoding="utf-8") as f: 
                    data = json.load(f)
                
                for qa_type in ['question','answer'] : 
                    emotion = data.get('dataSet', {}).get(qa_type,{}).get('emotion', [])
                    if not emotion : 
                        emotion_empty_count += 1
                    else :
                        emotion_count += 1

        except Exception as e : 
            print(f"{folder_name}이 비었습니다")
        print(f"{folder_name}을 종료!.")
    return (emotion_empty_count, emotion_count)

emotion_empty(training_path)
emotion_empty(valid_path)

## emotion이 어떻게 이뤄져있는지 파악

In [ ]:
training_path = "G:/내 드라이브/면접VR AI제작/extracted_TrainingData"
valid_path = "G:/내 드라이브/면접VR AI제작/extracted_Valid"


def kinds_of_emotion(base_path) : 
    emotion_kinds = []

    for folder_name in os.listdir(base_path) : 
        
        folder_path = os.path.join(base_path, folder_name)
        if not folder_path : 
            continue
        try :
            for file_name in os.listdir(folder_path) :
                if not file_name.endswith(".json") : 
                    continue
                file_path  = os.path.join(folder_path, file_name)
                
                with open(file_path, 'r', encoding = "utf-8") as f : 
                    data = json.load(f)
                
                for qa_type in ['question','answer'] : 
                    emotion = data.get("dataSet", {}).get(qa_type, {}).get("emotion", [])

                    for e in emotion : 
                        if not isinstance(e, dict) : 
                            continue
                        expr = e.get("expression", [])

                        if isinstance(expr, str) : 
                            expr = [expr]
                        elif not isinstance(expr, list) : 
                            continue

                        emotion_kinds.extend(expr)

        except Exception as e : 
            print(f"{folder_name}이 비었습니다")
                
    return emotion_kinds

def unique_emotion() : 
    train_emotions = kinds_of_emotion(training_path)
    valid_emotions = kinds_of_emotion(valid_path)

    total_emotions = train_emotions + valid_emotions

    unique_emotions = sorted(set(total_emotions))

    return unique_emotions

result = unique_emotion()
print(result)
print(len(result))

## emotion의 text별 expression연결하기

In [ ]:
# CSV로 저장
import csv

save_dir = "G:/내 드라이브/면접VR AI제작/Code/CSV모음"

def text_per_expression(base_path, save_name = "text_per_expression.csv") : 
    for folder_name in os.listdir(base_path) : 
        result = []
        folder_path = os.path.join(base_path, folder_name)
        if not folder_path : 
            continue
        try :
            for file_name in os.listdir(folder_path) :
                if not file_name.endswith(".json") : 
                    continue
                file_path  = os.path.join(folder_path, file_name)
                
                with open(file_path, 'r', encoding = "utf-8") as f : 
                    data = json.load(f)
                
                for qa_type in ['question','answer'] : 
                    emotion = data.get("dataSet", {}).get(qa_type, {}).get("emotion", [])

                    for e in emotion : 
                        if not isinstance(e, dict) : 
                            continue
                        text = e.get("text", "").strip()
                        if not text : 
                            continue

                        expr = e.get("expression", [])

                        if isinstance(expr, str) : 
                            expr = [expr]
                        elif not isinstance(expr, list) : 
                            continue
                        
                        result.append("text", ",".join(expr))
                        
        except Exception as e : 
            print(f"{folder_name}이 비었습니다")

    save_path = os.path.join(save_dir, save_name) 
    with open(save_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["text", "expressions"])
        writer.writerows(result)

    return save_path


## p- : positive, n- : negative, u- uncertain으로 명명

In [1]:
import pandas as pd
import numpy as np 

data = pd.read_csv("/home/j-k13b204/S13P31B204/model_test/Code/csv_collection/emotion_grouping.csv")

data.head()

,text,expression,sentiment
0,멀티 채널은 기존에 우리가 사용하고 있는 채널을 말하는데 어 쉽게 이야기하면 티비나...,u-fact,uncertain
1,반면에 옴니 채널은 온 오프라인 모바일 등 다양한 채널을 통해서 상품을 검색하고 구...,u-fact,uncertain
2,예를 들어 스마트폰 근거리 통신 기술을 이용해서 마트 앞을 지나는 고객에게 쿠폰을 ...,u-fact,uncertain
3,두 개에 차이점은 멀티 채널은 운영하고 있는 각 채널을 독립적으로 운영해서 온 오프...,u-fact,uncertain
4,제가 중국에서 일 년 정도 유학 생활을 했었는데요.,u-fact,uncertain


In [2]:
data['expression'].unique()

array(['u-fact', 'p-happiness', 'p-interest', 'p-affection',
       'p-achievement', 'n-anxiety', 'n-distress', 'u-belief',
       'n-sadness', 'p-gratitude', 'n-anger', 'n-shame'], dtype=object)

### sentiment에 업데이트

In [ ]:
data["sentiment"] = (
    data["expression"]
      .astype(str)
      .str.extract(r"^([unp])-", expand=False)
      .map({"u": "uncertain", "n": "negative", "p": "positive"})
      .fillna("unknown")
)

data.head()

In [ ]:
# csv파일 제작
data.to_csv("./CSV모음/emotion_csv/emotion_grouping.csv", index = False)

In [8]:
label_map = {"uncertain": 0, "negative": 1, "positive": 2}

data["label_encoding"] = (
    data["sentiment"]
        .map(label_map)            # 문자열 → 숫자 매핑
        .fillna(-1)                # 혹시 없는 값은 -1 처리 (unknown)
        .astype(int)
)

data['label_encoding'].value_counts()

label_encoding
0    9488
2    8068
1    2231
Name: count, dtype: int64

In [9]:
data.head()

,text,expression,sentiment,label_encoding
0,멀티 채널은 기존에 우리가 사용하고 있는 채널을 말하는데 어 쉽게 이야기하면 티비나...,u-fact,uncertain,0
1,반면에 옴니 채널은 온 오프라인 모바일 등 다양한 채널을 통해서 상품을 검색하고 구...,u-fact,uncertain,0
2,예를 들어 스마트폰 근거리 통신 기술을 이용해서 마트 앞을 지나는 고객에게 쿠폰을 ...,u-fact,uncertain,0
3,두 개에 차이점은 멀티 채널은 운영하고 있는 각 채널을 독립적으로 운영해서 온 오프...,u-fact,uncertain,0
4,제가 중국에서 일 년 정도 유학 생활을 했었는데요.,u-fact,uncertain,0


## 모델 양자화

In [ ]:
import os, re
import torch
import torch.nn as nn
from kobert_transformers import get_kobert_model, get_tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ===== 0) 감정 라벨 기본 이름 =====
# 실제 학습한 클래스 개수가 3개라면 예시: ['positive','neutral','negative']
# 모델에 맞게 자동으로 label_0, label_1,... 생성됨
def make_default_labels(num_classes: int):
    return {i: f"label_{i}" for i in range(num_classes)}

# ===== 1) 분류기(인퍼런스 전용) =====
class BertClassifier(nn.Module):
    def __init__(self, bert, hidden_size=768, num_classes=3, dr_rate=0.3):
        super().__init__()
        self.bert = bert
        self.dropout = nn.Dropout(dr_rate) if dr_rate and dr_rate > 0 else nn.Identity()
        self.classifier = nn.Linear(hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids)
        pooled = outputs.pooler_output if getattr(outputs, "pooler_output", None) is not None else outputs[0][:, 0]
        logits = self.classifier(self.dropout(pooled))
        return logits


def load_model(model_path: str, num_classes: int = 3, dr_rate: float = 0.3):
    """학습된 감정 모델 로드"""
    bert = get_kobert_model()
    model = BertClassifier(bert=bert, hidden_size=768, num_classes=num_classes, dr_rate=dr_rate)
    state = torch.load(model_path, map_location="cpu")

    # 불필요 키 정리
    for k in list(state.keys()):
        if k.startswith("loss_fn"):
            del state[k]

    model.load_state_dict(state, strict=False)
    model.eval().to(device)
    return model


def split_sentences(text: str):
    text = re.sub(r"\s+", " ", text.strip())
    sents = re.split(r"(?<=[.!?])\s+", text)
    return [s.strip() for s in sents if s.strip()]


@torch.no_grad()
def predict_emotion(model, tokenizer, sentence: str, label_map: dict[int, str], max_len: int = 128):
    """단일 문장의 감정 예측"""
    enc = tokenizer(
        sentence,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="pt",
    )
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)
    token_type_ids = enc.get("token_type_ids")
    token_type_ids = token_type_ids.to(device) if token_type_ids is not None else None

    logits = model(input_ids, attention_mask, token_type_ids)
    probs = torch.softmax(logits, dim=-1)[0]
    pred_idx = int(torch.argmax(probs).item())
    label = label_map.get(pred_idx, f"label_{pred_idx}")
    return label, float(probs[pred_idx].item()), probs.cpu().numpy()


def analyze_paragraph(model, tokenizer, paragraph: str, label_map: dict[int, str], max_len: int = 128):
    sents = split_sentences(paragraph)
    print("=" * 80)
    print("🎭 감정 분석 결과")
    print("=" * 80)
    print(f"\n총 {len(sents)}개의 문장이 발견되었습니다.\n")

    results = []
    for i, s in enumerate(sents, 1):
        label, conf, probs = predict_emotion(model, tokenizer, s, label_map, max_len=max_len)
        results.append({"sentence_num": i, "sentence": s, "emotion": label, "confidence": conf, "probabilities": probs})

        print(f"[문장 {i}]")
        print(f"내용: {s}")
        print(f"감정: {label} (확신도: {conf:.2%})")
        print("세부 확률:")
        for idx in range(len(probs)):
            print(f"  - {label_map.get(idx, f'label_{idx}')}: {probs[idx]:.2%}")
        print("-" * 80)
    return results


def print_summary(results):
    print("\n" + "=" * 80)
    print("📊 감정 요약")
    print("=" * 80)
    counts = {}
    for r in results:
        counts[r["emotion"]] = counts.get(r["emotion"], 0) + 1
    print("\n감정별 문장 수:")
    for k, v in sorted(counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {k}: {v}개 ({v / len(results) * 100:.1f}%)")
    avg_conf = sum(r["confidence"] for r in results) / max(1, len(results))
    print(f"\n평균 확신도: {avg_conf:.2%}")


if __name__ == "__main__":
    # 감정 모델 경로
    MODEL_PATH = "/home/j-k13b204/S13P31B204/model_test/kobert_emotion_model/model.pt"
    NUM_CLASSES = 3   # 예: 긍정·중립·부정 (학습 시 클래스 수로 변경) 0 : 
    MAX_LEN = 128

    print("모델/토크나이저 로드 중...")
    tokenizer = get_tokenizer()
    model = load_model(MODEL_PATH, num_classes=NUM_CLASSES, dr_rate=0.3)
    label_map = make_default_labels(NUM_CLASSES)
    print("✓ 로드 완료\n")

    paragraph = """
    면접을 준비하면서 긴장감도 있었지만, 제 강점을 충분히 보여줄 수 있어서 뿌듯했습니다.
    어려운 질문도 있었지만 차분하게 답변하려고 노력했고, 결과적으로 만족스러운 경험이었습니다.
    """

    results = analyze_paragraph(model, tokenizer, paragraph, label_map, max_len=MAX_LEN)
    print_summary(results)
# label_map = {0: "부정", 1: "중립", 2: "긍정"}

모델/토크나이저 로드 중...


✓ 로드 완료

🎭 감정 분석 결과

총 2개의 문장이 발견되었습니다.

[문장 1]
내용: 면접을 준비하면서 긴장감도 있었지만, 제 강점을 충분히 보여줄 수 있어서 뿌듯했습니다.
감정: label_0 (확신도: 38.62%)
세부 확률:
  - label_0: 38.62%
  - label_1: 37.31%
  - label_2: 24.07%
--------------------------------------------------------------------------------
[문장 2]
내용: 어려운 질문도 있었지만 차분하게 답변하려고 노력했고, 결과적으로 만족스러운 경험이었습니다.
감정: label_1 (확신도: 38.43%)
세부 확률:
  - label_0: 36.82%
  - label_1: 38.43%
  - label_2: 24.76%
--------------------------------------------------------------------------------

📊 감정 요약

감정별 문장 수:
  label_0: 1개 (50.0%)
  label_1: 1개 (50.0%)

평균 확신도: 38.52%
